In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [4]:
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [5]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [11]:
input_size = 28
hidden_size = 128
output_size = 20
num_layers = 2
num_epochs = 10
learning_rate = 0.001

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RNNClassifier(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.squeeze(1) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


In [14]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.squeeze(1)  
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [15]:
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 3.004592
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.292723
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.868081
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.901653
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.556153
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.561303
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.405054
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.322630
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.395681
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.215545

Test set: Average loss: 0.0004, Accuracy: 8908/10000 (89.08%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.399033
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.349294
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.189396
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.486420
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.206871
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.602037
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.118657
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.087921
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.19183